In [2811]:
import pandas as pd
import os
from flask import Flask, request, jsonify
import joblib
from dotenv import load_dotenv
from pymongo import MongoClient
import pandas as pd
from datetime import datetime


df = pd.read_csv("../notebooks/data/Ana Health_Tabela Modelo Previsão Churn - Tabela.csv")

In [2812]:
dfzzzz = pd.read_csv("../notebooks/data/data-engineering.csv")

In [2813]:
dfppp = pd.read_csv("../api/saida.csv")

In [2814]:
dfppp.head()

,PESSOA_PIPEDRIVE_notes_count,PESSOA_PIPEDRIVE_done_activities_count,FUNIL_ONBOARDING_PIPEDRIVE_activities_count,ATENDIMENTOS_AGENDA_Faltas Todos Atendimento,ATENDIMENTOS_AGENDA_Qde Atendimento Médico,ATENDIMENTOS_AGENDA_Faltas Atendimento Médico,ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento,ATENDIMENTOS_AGENDA_Faltas Acolhimento,ATENDIMENTOS_AGENDA_Qde Psicoterapia,ATENDIMENTOS_AGENDA_Qde Prescrições,...,Status_open,Status_won,lost_reason_Outro.1,lost_reason_[Associade] Cancelou assinatura,lost_reason_[Onboarding] Não retornou aos contatos de resgate,lost_reason_[Onboarding] Não tem interesse em seguir nas etapas do onboarding,stage_Boas-vindas,stage_Não iniciado,stage_Primeira reunião,stage_Questionário
0,0,7,3,0,0,0,1,0,0,0.0,...,False,False,False,False,True,False,False,False,False,True


In [2815]:
dfzzzz.head()

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_notes_count,PESSOA_PIPEDRIVE_done_activities_count,FUNIL_ONBOARDING_PIPEDRIVE_activities_count,ATENDIMENTOS_AGENDA_Faltas Todos Atendimento,ATENDIMENTOS_AGENDA_Qde Atendimento Médico,ATENDIMENTOS_AGENDA_Faltas Atendimento Médico,ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento,ATENDIMENTOS_AGENDA_Faltas Acolhimento,ATENDIMENTOS_AGENDA_Qde Psicoterapia,...,Status_open,Status_won,lost_reason_Outro.1,lost_reason_[Associade] Cancelou assinatura,lost_reason_[Onboarding] Não retornou aos contatos de resgate,lost_reason_[Onboarding] Não tem interesse em seguir nas etapas do onboarding,stage_Boas-vindas,stage_Não iniciado,stage_Primeira reunião,stage_Questionário
0,483,3,10,0,0,0,0,2,0,0,...,False,True,True,False,False,False,False,False,False,True
1,1207,0,9,0,0,0,0,1,0,0,...,False,True,True,False,False,False,False,False,False,True
2,1265,1,14,9,0,0,0,1,0,0,...,False,False,False,False,True,False,False,False,False,True
3,1318,0,11,5,0,0,0,1,0,0,...,False,True,True,False,False,False,False,False,False,True
4,1434,0,7,1,0,0,0,1,0,25,...,False,True,True,False,False,False,False,False,False,True


In [2816]:

dfzzzz = pd.DataFrame(dfzzzz.iloc[50]).T

In [2817]:
dfzzzz = dfzzzz.reset_index(drop=True).drop(columns=["PESSOA_PIPEDRIVE_id_person","stay_time"])

In [2818]:
dfppp.equals(dfzzzz)

False

In [2819]:
dfzzzz.to_json(force_ascii=False, orient='records', index=False)

'[{"PESSOA_PIPEDRIVE_notes_count":0,"PESSOA_PIPEDRIVE_done_activities_count":7,"FUNIL_ONBOARDING_PIPEDRIVE_activities_count":3,"ATENDIMENTOS_AGENDA_Faltas Todos Atendimento":0,"ATENDIMENTOS_AGENDA_Qde Atendimento Médico":0,"ATENDIMENTOS_AGENDA_Faltas Atendimento Médico":0,"ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento":1,"ATENDIMENTOS_AGENDA_Faltas Acolhimento":0,"ATENDIMENTOS_AGENDA_Qde Psicoterapia":0,"ATENDIMENTOS_AGENDA_Qde Prescrições":0.0,"TWILIO_Mensagens Inbound":29,"TWILIO_Mensagens Outbound":42,"TWILIO_Ligações Outbound":2,"PESSOA_PIPEDRIVE_age":27,"PESSOA_PIPEDRIVE_has_public_health_plan":1,"PESSOA_PIPEDRIVE_tem_data_dataframe_de_termino_de_contrato":1,"PESSOA_PIPEDRIVE_Tem_Canal_de_Preferência":0,"PESSOA_PIPEDRIVE_has_notes":0,"WHOQOL_Físico_New":0.5414887639,"WHOQOL_Psicológico_New":0.7004690034,"WHOQOL_Social_New":0.4268099387,"WHOQOL_Ambiental_New":4.0,"COMUNICARE_Problemas Abertos Bool":1,"TWILIO_Data Última Mensagens Inbound Recente":0,"TWILIO_Data Última Mensagens 

In [2820]:
model = joblib.load(os.path.join("../notebooks/data/SVR_model.joblib"))

In [2821]:
import pandas as pd
from datetime import datetime
import numpy as np
import datetime as dt
from sqlalchemy import false
def column_label_to_index(col_label):
    col_index = 0
    for c in col_label.upper():
        col_index = col_index * 26 + (ord(c) - ord('A') + 1)
    return col_index - 1
def add_prefix_to_first_row(dataframe, prefix, start_col_label, end_col_label):
    start_col = column_label_to_index(start_col_label)
    end_col = column_label_to_index(end_col_label)
    for col in range(start_col, end_col + 1):
        dataframe.iat[0, col] = prefix + dataframe.iat[0, col]
def mergeHeader_Columns(data):
    add_prefix_to_first_row(data, 'PESSOA_PIPEDRIVE_', 'A', 'R')
    add_prefix_to_first_row(data, 'FUNIL_ASSINATURA_PIPEDRIVE_', 'S', 'X')
    add_prefix_to_first_row(data, 'FUNIL_ONBOARDING_PIPEDRIVE_', 'Y', 'AH')
    add_prefix_to_first_row(data, 'ATENDIMENTOS_AGENDA_', 'AI', 'AU')
    add_prefix_to_first_row(data, 'WHOQOL_', 'AV', 'AZ')
    add_prefix_to_first_row(data, 'COMUNICARE_', 'BA', 'BA')
    add_prefix_to_first_row(data, 'TWILIO_', 'BB', 'BI')
    add_prefix_to_first_row(data, 'COBRANÇA_VINDI_', 'BJ', 'BQ')
    data.columns = data.iloc[0]
    data = data.drop(data.index[0])
    return data
def preprocessing(data_dataframe):
    from sklearn.impute import SimpleImputer

    # data_dataframe = mergeHeader_Columns(data_dataframe)
    # preprocessing 1
    data_dataframe = data_dataframe.drop(["PESSOA_PIPEDRIVE_id_person_recommendation","PESSOA_PIPEDRIVE_Recebe Comunicados?", "PESSOA_PIPEDRIVE_Interesses", "PESSOA_PIPEDRIVE_Pontos de Atenção", "FUNIL_ONBOARDING_PIPEDRIVE_id_label"], axis=1)
    data_dataframe['PESSOA_PIPEDRIVE_birthdate'] = pd.to_datetime(data_dataframe['PESSOA_PIPEDRIVE_birthdate'])
    data_dataframe['PESSOA_PIPEDRIVE_age'] = data_dataframe['PESSOA_PIPEDRIVE_birthdate'].apply(
        lambda x: datetime.today().year - x.year - ((datetime.today().month, datetime.today().day) < (x.month, x.day))
    )
    imputer = SimpleImputer(strategy='mean')
    data_dataframe['PESSOA_PIPEDRIVE_age'] = imputer.fit_transform(data_dataframe['PESSOA_PIPEDRIVE_age'].values.reshape(-1, 1))
    data_dataframe['PESSOA_PIPEDRIVE_age'] = np.round(data_dataframe['PESSOA_PIPEDRIVE_age']).astype(int)
    data_dataframe = data_dataframe.drop(['PESSOA_PIPEDRIVE_birthdate'], axis=1 )
    rows_to_drop = data_dataframe[data_dataframe["PESSOA_PIPEDRIVE_id_gender"].isin([117,110,111])]
    data_dataframe = data_dataframe.drop(rows_to_drop.index, axis=0)
    data_dataframe["PESSOA_PIPEDRIVE_id_gender"].fillna(64, inplace=True)
    data_dataframe["PESSOA_PIPEDRIVE_id_marrital_status"].fillna(80, inplace=True)
    
    data_dataframe["PESSOA_PIPEDRIVE_state"].fillna(data_dataframe["PESSOA_PIPEDRIVE_state"].mode()[0], inplace=True)
    data_dataframe["PESSOA_PIPEDRIVE_city"].fillna(data_dataframe["PESSOA_PIPEDRIVE_city"].mode()[0], inplace=True)
    
    def convert_to_int(x):
        if pd.notna(x) and int(x) == 412:
            return 1
        else:
            return 0

    
    mode_value = data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].mode()
    if not mode_value.empty:
        data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].fillna(mode_value[0], inplace=True)
    data_dataframe['PESSOA_PIPEDRIVE_has_public_health_plan'] = data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].apply(convert_to_int)
    
    data_dataframe = data_dataframe.drop(['PESSOA_PIPEDRIVE_id_health_plan'], axis=1)
    data_dataframe["PESSOA_PIPEDRIVE_tem_data_dataframe_de_termino_de_contrato"] = data_dataframe["PESSOA_PIPEDRIVE_contract_end_date"].apply(lambda x: 0 if pd.isna(x) else 1)
    data_dataframe.drop(["PESSOA_PIPEDRIVE_id_continuity_pf"], axis=1, inplace=True)
    data_dataframe["PESSOA_PIPEDRIVE_Canal de Preferência"].fillna(0, inplace=True)
    data_dataframe["PESSOA_PIPEDRIVE_Tem_Canal_de_Preferência"] = data_dataframe["PESSOA_PIPEDRIVE_Canal de Preferência"].apply(lambda x: 1 if int(x) > 0  else 0)
    data_dataframe["PESSOA_PIPEDRIVE_has_notes"] = data_dataframe["PESSOA_PIPEDRIVE_notes_count"].apply(lambda x: 1 if int(x) > 0 else 0)
    
    # preprocessing 2
    data_dataframe = data_dataframe.drop(["ATENDIMENTOS_AGENDA_Faltas Psicoterapia","TWILIO_Ligações Inbound", "TWILIO_Data Última Ligações Inbound","COBRANÇA_VINDI_Qde Total de Faturas","COBRANÇA_VINDI_Qde Total de Tentativas de Cobrança","COBRANÇA_VINDI_Método de Pagamento","COBRANÇA_VINDI_Valor Médio da Mensalidade","COBRANÇA_VINDI_Qde Total de Faturas Pagas após Vencimento","COBRANÇA_VINDI_Qde Total de Faturas Inadimpletes","COBRANÇA_VINDI_Valor Total Inadimplência"], axis=1)
    data_dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].fillna(0, inplace=True)
    

    data_dataframe = data_dataframe.drop(columns="WHOQOL_Qde Respostas WHOQOL")
    data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico"].str.split(';').str[-1].str.strip()
    data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico_New"].str.replace(",",".")
    data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico_New"].astype(float) 
    data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico_New"].fillna(data_dataframe["WHOQOL_Físico_New"].median())
    data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico"].str.split(';').str[-1].str.strip()
    data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico_New"].str.replace(",",".")
    data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico_New"].astype(float) 
    data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico_New"].fillna(data_dataframe["WHOQOL_Psicológico_New"].median())
    data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social"].str.split(';').str[-1].str.strip()
    data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social_New"].str.replace(",",".")
    data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social_New"].astype(float) 
    data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social_New"].fillna(data_dataframe["WHOQOL_Social_New"].median())
    data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental"].str.split(';').str[-1].str.strip()
    data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental_New"].str.replace(",",".")
    data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental_New"].astype(float) 
    data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental_New"].fillna(data_dataframe["WHOQOL_Ambiental_New"].median())
    data_dataframe["COMUNICARE_Problemas Abertos Bool"] = data_dataframe["COMUNICARE_Problemas Abertos"].notnull().astype(int)
    data_dataframe["COMUNICARE_Problemas Abertos Bool"].fillna(0, inplace=True)
    data_dataframe["TWILIO_Mensagens Inbound"].fillna(0, inplace=True)
    data_dataframe["TWILIO_Data Última Mensagens Inbound"] = pd.to_datetime(data_dataframe["TWILIO_Data Última Mensagens Inbound"])
    data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] = datetime.now() - data_dataframe["TWILIO_Data Última Mensagens Inbound"]
    data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].fillna('', inplace=True)
    data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] = data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].astype(str)
    data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] = data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].str.extract('(\d+) days').astype(float)
    data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"] = data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] < data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].median()
    data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"].fillna(0)
    data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"].replace(True, 1, inplace=True)
    data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"].replace(False, 0, inplace=True)
    data_dataframe = data_dataframe.drop(columns=["WHOQOL_Ambiental","WHOQOL_Social","WHOQOL_Físico","WHOQOL_Psicológico","COMUNICARE_Problemas Abertos","TWILIO_Data Última Mensagens Inbound","ATENDIMENTOS_AGENDA_Datas Psicoterapia","TWILIO_Data Última Mensagens Inbound Tempo Passado"])
    
    
    # preprocessing 3
    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        if pd.notna(valor) == False: 
            if pd.notna(data_dataframe.loc[indice, "PESSOA_PIPEDRIVE_contract_end_date"]):
                data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "PESSOA_PIPEDRIVE_contract_end_date"]
    data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].fillna(dt.date.today(), inplace=True)
    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        index = str(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"]).find(";")
        if index != -1:
            data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"][:index]
    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        tamanho = len(str(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"]))
        if tamanho > 10:
            data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"][:10]	
    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = pd.to_datetime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], format='%Y-%m-%d', errors='coerce')
        data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].strftime('%Y-%m-%d')
    tempo_permanencia = []
    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"].items():
        if pd.notna(valor):
            index = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"].find(";")
            if index != -1:
                data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"][:index]
    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        if pd.notna(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"]):
            tempo_1 = datetime.strptime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], '%Y-%m-%d')
            tempo_2 = datetime.strptime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"], '%Y-%m-%d')
            tempo_permanencia.append(str(tempo_1 - tempo_2))
        else:
            tempo_1 = datetime.strptime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], '%Y-%m-%d')
            tempo_2 = datetime.strptime(data_dataframe.loc[indice, "PESSOA_PIPEDRIVE_contract_start_date"], '%Y-%m-%d')
            tempo_permanencia.append(str(tempo_1 - tempo_2))
    data_dataframe['stay_time'] = tempo_permanencia
    for indice, valor in data_dataframe["stay_time"].items():
        index = data_dataframe.loc[indice, "stay_time"].find(",")
        if index != -1:
            data_dataframe.loc[indice, "stay_time"] = data_dataframe.loc[indice, "stay_time"][:index]
    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].items():
        if pd.notna(valor):  
            index = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].find(";")
            if index != -1: 
                data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"][:index]
        else:
            data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"] = "Assinatura ativa"
            
        contagem = data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].value_counts()
        agrupamento = contagem[contagem < 20].index
        data_dataframe.loc[data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].isin(agrupamento), "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"] = "Outro"
    data_dataframe['FUNIL_ONBOARDING_PIPEDRIVE_add_time'].fillna('Não iniciado', inplace=True)
    ultimos_estados = []
    for indice, valor in data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_add_time"].items():
        if valor != "Não iniciado":
            if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_welcome"]):
                if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_first_meeting"]):
                    if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_whoqol"]):
                        ultimos_estados.append("Questionário")
                    else:
                        ultimos_estados.append("Primeira reunião")
                else:
                    ultimos_estados.append("Boas-vindas")
            
        else:
            ultimos_estados.append("Não iniciado")
    data_dataframe["last_stage_concluded"] = ultimos_estados
    data_dataframe.drop(["FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_welcome", "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_first_meeting", "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_whoqol"], axis=1, inplace=True)
    data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_status"].fillna("Não iniciado", inplace=True)
    tempo = []
    for indice, valor in data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_add_time"].items():
        if valor != "Não iniciado":
            if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"]) == True:
                tempo.append(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"])
            elif pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_won_time"]) == True:
                tempo.append(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_won_time"])
            else:
                tempo.append("Em aberto")
        else:
            tempo.append("Não iniciado")
    data_dataframe['process_time'] = tempo
    data_dataframe.drop(["FUNIL_ONBOARDING_PIPEDRIVE_won_time", "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"], axis=1, inplace=True)
    data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_activities_count"] = data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_activities_count"].fillna(0)
    data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimento Médico"] = data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimento Médico"].fillna(0)
    data_dataframe["ATENDIMENTOS_AGENDA_Faltas Atendimento Médico"] = data_dataframe["ATENDIMENTOS_AGENDA_Faltas Atendimento Médico"].fillna(0)
    data_dataframe["ATENDIMENTOS_AGENDA_Datas Atendimento Médico"] = data_dataframe["ATENDIMENTOS_AGENDA_Datas Atendimento Médico"].fillna("Nunca ocorreu")
    for indice, valor in data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"].items():
        if pd.notna(valor) == False:
            if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_status"]) == False and pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_add_time"]):
                data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo em aberto"
            if data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_status"] == "won":
                data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo concluído"
            else:
                data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo não iniciado"
        contagem = data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"].value_counts()
        agrupamento = contagem[contagem < 23].index
        data_dataframe.loc[data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"].isin(agrupamento), "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Outro"
    data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento"] = data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento"].fillna(0)
    data_dataframe["ATENDIMENTOS_AGENDA_Faltas Acolhimento"] = data_dataframe["ATENDIMENTOS_AGENDA_Faltas Acolhimento"].fillna(0)
    data_dataframe["ATENDIMENTOS_AGENDA_Datas Acolhimento"] = data_dataframe["ATENDIMENTOS_AGENDA_Datas Acolhimento"].fillna("Nunca ocorreu")
    data_dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] = data_dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].fillna(0)
    # preprocessing 4
    
    data_dataframe["TWILIO_Mensagens Outbound"].fillna(0, inplace=True)
    data_dataframe["TWILIO_Data Última Mensagens Outbound"] = pd.to_datetime(data_dataframe["TWILIO_Data Última Mensagens Outbound"])
    data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] = datetime.now() - data_dataframe["TWILIO_Data Última Mensagens Outbound"]
    data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].fillna('', inplace=True)
    
    data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].astype(str)
    data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].str.extract('(\d+) days').astype(float)
    data_dataframe = data_dataframe.drop(["TWILIO_Data Última Mensagens Outbound"], axis=1)
    data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"] = data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] < data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].median()
    data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"].fillna(0)
    data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"].replace(True, 1, inplace=True)
    data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"].replace(False, 0, inplace=True)
    data_dataframe = data_dataframe.drop(["TWILIO_Data Última Mensagens Outbound Tempo passado"], axis=1)
    data_dataframe["TWILIO_Ligações Outbound"].fillna(0, inplace=True)
    data_dataframe["TWILIO_Data Última Ligações Outbound"] = pd.to_datetime(data_dataframe["TWILIO_Data Última Ligações Outbound"])
    
    data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] = datetime.now() - data_dataframe["TWILIO_Data Última Ligações Outbound"]
    data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].fillna('', inplace=True)
    data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].astype(str)
    data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].str.extract('(\d+) days').astype(float)
    data_dataframe = data_dataframe.drop(["TWILIO_Data Última Ligações Outbound"], axis=1)
    data_dataframe["TWILIO_Data Última Ligações Outbound Recente"] = data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] < data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].median()
    data_dataframe["TWILIO_Data Última Ligações Outbound Recente"].fillna(0)
    data_dataframe["TWILIO_Data Última Ligações Outbound Recente"].replace(True, 1, inplace=True)
    data_dataframe["TWILIO_Data Última Ligações Outbound Recente"].replace(False, 0, inplace=True)
    
    data_dataframe = data_dataframe.drop(["TWILIO_Data Última Ligações Outbound Tempo passado"], axis=1)
    data_dataframe = data_dataframe.drop(["COBRANÇA_VINDI_Qde Perfis de Pagamento Inativos"], axis=1)
    # preprocessing especials 
    
    data_dataframe = data_dataframe[data_dataframe['FUNIL_ASSINATURA_PIPEDRIVE_status'].isin(['won', 'lost'])]
    data_dataframe = data_dataframe.drop(data_dataframe[(data_dataframe['FUNIL_ASSINATURA_PIPEDRIVE_status'] == 'won') & (~data_dataframe['PESSOA_PIPEDRIVE_contract_end_date'].isnull())].index)
    data_dataframe = data_dataframe.drop(data_dataframe[(data_dataframe['FUNIL_ASSINATURA_PIPEDRIVE_status'] == 'lost') & (data_dataframe['PESSOA_PIPEDRIVE_contract_end_date'].isnull())].index)
    #exportando df pronto
    
    return data_dataframe
    

In [2822]:
import pandas as pd
import numpy as np
from category_encoders import CountEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy import stats
from datetime import datetime
from sklearn.utils import resample
from sqlalchemy import false


def feature_engineering(data_pre):



        # feature engineering 1 + 3

            
    data_pre["stay_time"] = data_pre["stay_time"].str.extract('(\d+) days').astype(float)

    data_pre["stay_time"].fillna(0,inplace=True)

    data_pre["ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento"].fillna(0,inplace=True)
    data_pre["ATENDIMENTOS_AGENDA_Faltas Acolhimento"].fillna(0,inplace=True)
    data_pre["ATENDIMENTOS_AGENDA_Datas Acolhimento"].fillna("Nunca ocorreu",inplace=True)
    
    data_pre = data_pre[~data_pre['ATENDIMENTOS_AGENDA_Datas Acolhimento'].astype(str).str.contains("Nunca ocorreu")]

    data_pre = data_pre.explode('ATENDIMENTOS_AGENDA_Datas Acolhimento')

    data_pre["TWILIO_Mensagens Já Enviou"] = int(data_pre["TWILIO_Mensagens Inbound"].iloc[0]) > 0
    data_pre["TWILIO_Mensagens Razão"] = int(data_pre["TWILIO_Mensagens Outbound"].iloc[0]) / int(data_pre["TWILIO_Mensagens Inbound"].where(data_pre["TWILIO_Mensagens Já Enviou"], 1).iloc[0])
    
    data_pre["PESSOA_PIPEDRIVE CRIANÇA"] = data_pre["PESSOA_PIPEDRIVE_age"] <= 16
    data_pre["PESSOA_PIPEDRIVE CRIANÇA"].fillna(0)
    data_pre["PESSOA_PIPEDRIVE CRIANÇA"].replace(True, 1, inplace=True)
    data_pre["PESSOA_PIPEDRIVE CRIANÇA"].replace(False, 0, inplace=True)

    data_pre["PESSOA_PIPEDRIVE JOVEM"] = (data_pre["PESSOA_PIPEDRIVE_age"] > 16) & (data_pre["PESSOA_PIPEDRIVE_age"] <= 30)
    data_pre["PESSOA_PIPEDRIVE JOVEM"].fillna(0)
    data_pre["PESSOA_PIPEDRIVE JOVEM"].replace(True, 1, inplace=True)
    data_pre["PESSOA_PIPEDRIVE JOVEM"].replace(False, 0, inplace=True)
    
    data_pre["PESSOA_PIPEDRIVE ADULTO"] = (data_pre["PESSOA_PIPEDRIVE_age"] > 30) & (data_pre["PESSOA_PIPEDRIVE_age"] <= 60)
    data_pre["PESSOA_PIPEDRIVE ADULTO"].fillna(0)
    data_pre["PESSOA_PIPEDRIVE ADULTO"].replace(True, 1, inplace=True)
    data_pre["PESSOA_PIPEDRIVE ADULTO"].replace(False, 0, inplace=True)


    data_pre["PESSOA_PIPEDRIVE IDOSO"] = data_pre["PESSOA_PIPEDRIVE_age"] > 60
    data_pre["PESSOA_PIPEDRIVE IDOSO"].fillna(0)
    data_pre["PESSOA_PIPEDRIVE IDOSO"].replace(True, 1, inplace=True)
    data_pre["PESSOA_PIPEDRIVE IDOSO"].replace(False, 0, inplace=True)
    

    # data_pre["TWILIO_Ligações Outbound Qtd Significativa"] = int(data_pre["TWILIO_Ligações Outbound"]) >= data_pre["TWILIO_Ligações Outbound"].mean()
    # data_pre["TWILIO_Ligações Outbound Qtd Significativa"].fillna(0)
    # data_pre["TWILIO_Ligações Outbound Qtd Significativa"].replace(True, 1, inplace=True)
    # data_pre["TWILIO_Ligações Outbound Qtd Significativa"].replace(False, 0, inplace=True)

    data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Nenhum"] = data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] == 0
    data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Nenhum"].fillna(0)
    data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Nenhum"].replace(True, 1, inplace=True)
    data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Nenhum"].replace(False, 0, inplace=True)

    # data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Pouco"] = (data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] > 0) & (data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] <= data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].mean() )
    # data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Pouco"].fillna(0)
    # data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Pouco"].replace(True, 1, inplace=True)
    # data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Pouco"].replace(False, 0, inplace=True)

    # data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Muito"] = data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] > data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].mean() 
    # data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Muito"].fillna(0)
    # data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Muito"].replace(True, 1, inplace=True)
    # data_pre["ATENDIMENTOS_AGENDA_Qde Psicoterapia Muito"].replace(False, 0, inplace=True)

    data_pre["ATENDIMENTOS_AGENDA_Qde Prescrições"].fillna(0,inplace=True)

    data_pre = data_pre.drop(["ATENDIMENTOS_AGENDA_Datas Prescrição"],axis=1)
    

    
    data_pre = data_pre[data_pre["PESSOA_PIPEDRIVE_id_gender"].isin(['64','63'])]   
    data_pre["PESSOA_PIPEDRIVE_id_gender Binário"] = data_pre["PESSOA_PIPEDRIVE_id_gender"].map({'64': 0, '63': 1})
    
    data_pre = data_pre[data_pre["FUNIL_ASSINATURA_PIPEDRIVE_id_stage"].isin(['65','64'])]   
    data_pre["FUNIL_ASSINATURA_PIPEDRIVE_id_stage Binário"] = data_pre["FUNIL_ASSINATURA_PIPEDRIVE_id_stage"].map({'65': 0, '64': 1})

    

    data_pre = pd.get_dummies(data_pre, columns=['PESSOA_PIPEDRIVE_id_marrital_status'], prefix='Status')
    data_pre = pd.get_dummies(data_pre, columns=['PESSOA_PIPEDRIVE_state'], prefix='Estado')

    ce = CountEncoder()
    data_pre['PESSOA_PIPEDRIVE_city Codificada'] = ce.fit_transform(data_pre['PESSOA_PIPEDRIVE_city'])


    data_pre = data_pre.drop(columns=["PESSOA_PIPEDRIVE_postal_code","PESSOA_PIPEDRIVE_id_person","PESSOA_PIPEDRIVE_city","PESSOA_PIPEDRIVE_id_gender","PESSOA_PIPEDRIVE_contract_start_date","PESSOA_PIPEDRIVE_contract_end_date","FUNIL_ASSINATURA_PIPEDRIVE_id_stage","FUNIL_ASSINATURA_PIPEDRIVE_id_org","FUNIL_ASSINATURA_PIPEDRIVE_lost_time","FUNIL_ASSINATURA_PIPEDRIVE_start_of_service","FUNIL_ONBOARDING_PIPEDRIVE_add_time","ATENDIMENTOS_AGENDA_Datas Atendimento Médico","ATENDIMENTOS_AGENDA_Datas Acolhimento","process_time"])

    


    # feature engineering 2 + 3

    data_pre = pd.get_dummies(data_pre,columns=["FUNIL_ASSINATURA_PIPEDRIVE_status"], prefix='status')


    data_pre = pd.get_dummies(data_pre,columns=['FUNIL_ASSINATURA_PIPEDRIVE_lost_reason'], prefix='lost_reason')

    data_pre = pd.get_dummies(data_pre,columns=['PESSOA_PIPEDRIVE_Canal de Preferência'], prefix='canal_preferencia')    
    
    data_pre = pd.get_dummies(data_pre,columns=['FUNIL_ONBOARDING_PIPEDRIVE_status'], prefix='Status')


    data_pre = pd.get_dummies(data_pre,columns=['FUNIL_ONBOARDING_PIPEDRIVE_lost_reason'], prefix='lost_reason')


    data_pre.drop('ATENDIMENTOS_AGENDA_Qde Todos Atendimentos', axis='columns', inplace=True)
    

    
    # feature engineering 4 + 3

    if data_pre['WHOQOL_Físico_New'].dtype == 'object':
        # Aplicando codificação one-hot para variáveis categóricas
        encoder = OneHotEncoder()
        encoded = encoder.fit_transform(data_pre[['WHOQOL_Físico_New']])
        encoded_df = pd.data_pre(encoded.toarray(), columns=encoder.get_feature_names_out(['WHOQOL_Físico_New']))
        data_pre = data_pre.join(encoded_df)
    else:
        # Aplicando normalização ou padronização para variáveis numéricas
        scaler = StandardScaler()
        if data_pre['WHOQOL_Físico_New'].count() >= 2:
            data_pre['WHOQOL_Físico_New'] = scaler.fit_transform(data_pre[['WHOQOL_Físico_New']])
        else:
            # Lide com o caso de ter apenas uma linha ou todos os valores NaN
            data_pre['WHOQOL_Físico_New'] = data_pre['WHOQOL_Físico_New'].fillna(0)
    
    # Calculando o intervalo interquartil
    Q1 = data_pre['WHOQOL_Físico_New'].quantile(0.25)
    Q3 = data_pre['WHOQOL_Físico_New'].quantile(0.75)
    IQR = Q3 - Q1

    # Removendo os outliers
    data_pre = data_pre[(data_pre['WHOQOL_Físico_New'] >= Q1 - 1.5 * IQR) & (data_pre['WHOQOL_Físico_New'] <= Q3 + 1.5 * IQR)]



    # Verificando se é categórica ou numérica
    if data_pre['WHOQOL_Psicológico_New'].dtype == 'object':
        # Aplicando codificação one-hot para variáveis categóricas
        encoder = OneHotEncoder()
        encoded = encoder.fit_transform(data_pre[['WHOQOL_Psicológico_New']])
        encoded_df = pd.data_pre(encoded.toarray(), columns=encoder.get_feature_names_out(['WHOQOL_Psicológico_New']))
        data_pre = data_pre.join(encoded_df)
    else:
        # Aplicando normalização ou padronização para variáveis numéricas
        scaler = StandardScaler()
        if data_pre['WHOQOL_Psicológico_New'].count() >= 2:
            data_pre['WHOQOL_Psicológico_New'] = scaler.fit_transform(data_pre[['WHOQOL_Psicológico_New']])
        else:
            # Lide com o caso de ter apenas uma linha ou todos os valores NaN
            data_pre['WHOQOL_Psicológico_New'] = data_pre['WHOQOL_Psicológico_New'].fillna(0)

        # Calculando o intervalo interquartil
        Q1 = data_pre['WHOQOL_Psicológico_New'].quantile(0.25)
        Q3 = data_pre['WHOQOL_Psicológico_New'].quantile(0.75)
        IQR = Q3 - Q1
        
        # Removendo os outliers
        data_pre = data_pre[(data_pre['WHOQOL_Psicológico_New'] >= Q1 - 1 * IQR) & (data_pre['WHOQOL_Psicológico_New'] <= Q3 + 1 * IQR)]
    
    # Verificando se é categórica ou numérica
    if data_pre['WHOQOL_Social_New'].dtype == 'object':
        # Aplicando codificação one-hot para variáveis categóricas
        encoder = OneHotEncoder()
        encoded = encoder.fit_transform(data_pre[['WHOQOL_Social_New']])
        encoded_df = pd.data_pre(encoded.toarray(), columns=encoder.get_feature_names_out(['WHOQOL_Social_New']))
        data = data.join(encoded_df)
    else:
        # Aplicando normalização ou padronização para variáveis numéricas
        scaler = StandardScaler()
        if data_pre['WHOQOL_Social_New'].count() >= 2:
            data_pre['WHOQOL_Social_New'] = scaler.fit_transform(data_pre[['WHOQOL_Social_New']])
        else:
            # Lide com o caso de ter apenas uma linha ou todos os valores NaN
            data_pre['WHOQOL_Social_New'] = data_pre['WHOQOL_Social_New'].fillna(0)

    Q1 = data_pre['WHOQOL_Social_New'].quantile(0.25)
    Q3 = data_pre['WHOQOL_Social_New'].quantile(0.75)
    IQR = Q3 - Q1

    # Removendo os outliers
    data_pre = data_pre[(data_pre['WHOQOL_Social_New'] >= Q1 - 1 * IQR) & (data_pre['WHOQOL_Social_New'] <= Q3 + 1 * IQR)]
    data_pre = pd.get_dummies(data_pre, columns=['last_stage_concluded'], prefix='stage')
    print(data_pre.shape)

    
    return data_pre







In [2823]:
ddd = pd.read_csv("../notebooks/data/data-preprocessed.csv")

In [2824]:
ddd["WHOQOL_Físico_New"]

0      4.0
1      3.0
2      2.0
3      3.9
4      3.7
      ... 
967    3.7
968    2.9
969    3.7
970    3.7
971    3.7
Name: WHOQOL_Físico_New, Length: 972, dtype: float64

In [2825]:
data_pfe.head()

,PESSOA_PIPEDRIVE_notes_count,PESSOA_PIPEDRIVE_done_activities_count,FUNIL_ONBOARDING_PIPEDRIVE_activities_count,ATENDIMENTOS_AGENDA_Faltas Todos Atendimento,ATENDIMENTOS_AGENDA_Qde Atendimento Médico,ATENDIMENTOS_AGENDA_Faltas Atendimento Médico,ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento,ATENDIMENTOS_AGENDA_Faltas Acolhimento,ATENDIMENTOS_AGENDA_Qde Psicoterapia,ATENDIMENTOS_AGENDA_Qde Prescrições,...,TWILIO_Mensagens Já Enviou,TWILIO_Mensagens Razão,PESSOA_PIPEDRIVE CRIANÇA,PESSOA_PIPEDRIVE JOVEM,PESSOA_PIPEDRIVE ADULTO,PESSOA_PIPEDRIVE IDOSO,ATENDIMENTOS_AGENDA_Qde Psicoterapia Nenhum,PESSOA_PIPEDRIVE_id_gender Binário,FUNIL_ASSINATURA_PIPEDRIVE_id_stage Binário,PESSOA_PIPEDRIVE_city Codificada


In [2826]:
df2 = pd.read_csv("../notebooks/data/data-engineering.csv")

df2.head()

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_notes_count,PESSOA_PIPEDRIVE_done_activities_count,FUNIL_ONBOARDING_PIPEDRIVE_activities_count,ATENDIMENTOS_AGENDA_Faltas Todos Atendimento,ATENDIMENTOS_AGENDA_Qde Atendimento Médico,ATENDIMENTOS_AGENDA_Faltas Atendimento Médico,ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento,ATENDIMENTOS_AGENDA_Faltas Acolhimento,ATENDIMENTOS_AGENDA_Qde Psicoterapia,...,Status_open,Status_won,lost_reason_Outro.1,lost_reason_[Associade] Cancelou assinatura,lost_reason_[Onboarding] Não retornou aos contatos de resgate,lost_reason_[Onboarding] Não tem interesse em seguir nas etapas do onboarding,stage_Boas-vindas,stage_Não iniciado,stage_Primeira reunião,stage_Questionário
0,483,3,10,0,0,0,0,2,0,0,...,False,True,True,False,False,False,False,False,False,True
1,1207,0,9,0,0,0,0,1,0,0,...,False,True,True,False,False,False,False,False,False,True
2,1265,1,14,9,0,0,0,1,0,0,...,False,False,False,False,True,False,False,False,False,True
3,1318,0,11,5,0,0,0,1,0,0,...,False,True,True,False,False,False,False,False,False,True
4,1434,0,7,1,0,0,0,1,0,25,...,False,True,True,False,False,False,False,False,False,True


In [2827]:
df2.head()

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_notes_count,PESSOA_PIPEDRIVE_done_activities_count,FUNIL_ONBOARDING_PIPEDRIVE_activities_count,ATENDIMENTOS_AGENDA_Faltas Todos Atendimento,ATENDIMENTOS_AGENDA_Qde Atendimento Médico,ATENDIMENTOS_AGENDA_Faltas Atendimento Médico,ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento,ATENDIMENTOS_AGENDA_Faltas Acolhimento,ATENDIMENTOS_AGENDA_Qde Psicoterapia,...,Status_open,Status_won,lost_reason_Outro.1,lost_reason_[Associade] Cancelou assinatura,lost_reason_[Onboarding] Não retornou aos contatos de resgate,lost_reason_[Onboarding] Não tem interesse em seguir nas etapas do onboarding,stage_Boas-vindas,stage_Não iniciado,stage_Primeira reunião,stage_Questionário
0,483,3,10,0,0,0,0,2,0,0,...,False,True,True,False,False,False,False,False,False,True
1,1207,0,9,0,0,0,0,1,0,0,...,False,True,True,False,False,False,False,False,False,True
2,1265,1,14,9,0,0,0,1,0,0,...,False,False,False,False,True,False,False,False,False,True
3,1318,0,11,5,0,0,0,1,0,0,...,False,True,True,False,False,False,False,False,False,True
4,1434,0,7,1,0,0,0,1,0,25,...,False,True,True,False,False,False,False,False,False,True


In [2828]:
df2["stay_time"]

0      211.0
1      365.0
2      395.0
3      420.0
4      381.0
       ...  
561     38.0
562     32.0
563     37.0
564     31.0
565     31.0
Name: stay_time, Length: 566, dtype: float64

In [2829]:
df3 = pd.read_csv("../notebooks/data/Ana Health_Tabela Modelo Previsão Churn - Tabela.csv")

In [2830]:
df3 = mergeHeader_Columns(df3)

In [2831]:
df3_filtrado = df3[df3["PESSOA_PIPEDRIVE_id_person"] == '5121']


In [2832]:
df3_filtrado

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_birthdate,PESSOA_PIPEDRIVE_id_gender,PESSOA_PIPEDRIVE_id_marrital_status,PESSOA_PIPEDRIVE_state,PESSOA_PIPEDRIVE_city,PESSOA_PIPEDRIVE_postal_code,PESSOA_PIPEDRIVE_id_health_plan,PESSOA_PIPEDRIVE_id_person_recommendation,PESSOA_PIPEDRIVE_contract_start_date,...,TWILIO_Ligações Outbound,TWILIO_Data Última Ligações Outbound,COBRANÇA_VINDI_Qde Total de Faturas,COBRANÇA_VINDI_Qde Total de Tentativas de Cobrança,COBRANÇA_VINDI_Método de Pagamento,COBRANÇA_VINDI_Valor Médio da Mensalidade,COBRANÇA_VINDI_Qde Total de Faturas Pagas após Vencimento,COBRANÇA_VINDI_Qde Total de Faturas Inadimpletes,COBRANÇA_VINDI_Valor Total Inadimplência,COBRANÇA_VINDI_Qde Perfis de Pagamento Inativos
906,5121,1997-01-19,63,80,Amazonas,Novo Airão,69730-000,412,NaN,2023-06-12,...,2,2023-08-24 15:48:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2833]:
type(df3_filtrado["PESSOA_PIPEDRIVE_id_gender"].iloc[0])

str

In [2834]:
def verificar_tipo(valor):
    return type(valor)

# Aplicar a função a cada elemento do DataFrame
tipos_de_dados = df3_filtrado.applymap(verificar_tipo)

print(tipos_de_dados)

0   PESSOA_PIPEDRIVE_id_person PESSOA_PIPEDRIVE_birthdate  \
906              <class 'str'>              <class 'str'>   

0   PESSOA_PIPEDRIVE_id_gender PESSOA_PIPEDRIVE_id_marrital_status  \
906              <class 'str'>                       <class 'str'>   

0   PESSOA_PIPEDRIVE_state PESSOA_PIPEDRIVE_city PESSOA_PIPEDRIVE_postal_code  \
906          <class 'str'>         <class 'str'>                <class 'str'>   

0   PESSOA_PIPEDRIVE_id_health_plan PESSOA_PIPEDRIVE_id_person_recommendation  \
906                   <class 'str'>                           <class 'float'>   

0   PESSOA_PIPEDRIVE_contract_start_date  ... TWILIO_Ligações Outbound  \
906                        <class 'str'>  ...            <class 'str'>   

0   TWILIO_Data Última Ligações Outbound COBRANÇA_VINDI_Qde Total de Faturas  \
906                        <class 'str'>                     <class 'float'>   

0   COBRANÇA_VINDI_Qde Total de Tentativas de Cobrança  \
906                                    <cl

/var/folders/5b/y5xgth0n79lbt3pkmgmjphzh0000gn/T/ipykernel_2920/64875285.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tipos_de_dados = df3_filtrado.applymap(verificar_tipo)


In [2835]:
df3_filtrado.to_json(force_ascii=False, orient='records', index=False)

'[{"PESSOA_PIPEDRIVE_id_person":"5121","PESSOA_PIPEDRIVE_birthdate":"1997-01-19","PESSOA_PIPEDRIVE_id_gender":"63","PESSOA_PIPEDRIVE_id_marrital_status":"80","PESSOA_PIPEDRIVE_state":"Amazonas","PESSOA_PIPEDRIVE_city":"Novo Airão","PESSOA_PIPEDRIVE_postal_code":"69730-000","PESSOA_PIPEDRIVE_id_health_plan":"412","PESSOA_PIPEDRIVE_id_person_recommendation":null,"PESSOA_PIPEDRIVE_contract_start_date":"2023-06-12","PESSOA_PIPEDRIVE_contract_end_date":null,"PESSOA_PIPEDRIVE_id_continuity_pf":null,"PESSOA_PIPEDRIVE_Canal de Preferência":null,"PESSOA_PIPEDRIVE_notes_count":"2","PESSOA_PIPEDRIVE_done_activities_count":"9","PESSOA_PIPEDRIVE_Recebe Comunicados?":null,"PESSOA_PIPEDRIVE_Interesses":null,"PESSOA_PIPEDRIVE_Pontos de Atenção":null,"FUNIL_ASSINATURA_PIPEDRIVE_id_stage":"65","FUNIL_ASSINATURA_PIPEDRIVE_id_org":"2457","FUNIL_ASSINATURA_PIPEDRIVE_status":"won","FUNIL_ASSINATURA_PIPEDRIVE_start_of_service":"2023-06-12","FUNIL_ASSINATURA_PIPEDRIVE_lost_time":null,"FUNIL_ASSINATURA_PIPEDRI

In [2836]:
df3 = pd.DataFrame(df3.iloc[750]).T

In [2837]:
df3.head()

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_birthdate,PESSOA_PIPEDRIVE_id_gender,PESSOA_PIPEDRIVE_id_marrital_status,PESSOA_PIPEDRIVE_state,PESSOA_PIPEDRIVE_city,PESSOA_PIPEDRIVE_postal_code,PESSOA_PIPEDRIVE_id_health_plan,PESSOA_PIPEDRIVE_id_person_recommendation,PESSOA_PIPEDRIVE_contract_start_date,...,TWILIO_Ligações Outbound,TWILIO_Data Última Ligações Outbound,COBRANÇA_VINDI_Qde Total de Faturas,COBRANÇA_VINDI_Qde Total de Tentativas de Cobrança,COBRANÇA_VINDI_Método de Pagamento,COBRANÇA_VINDI_Valor Médio da Mensalidade,COBRANÇA_VINDI_Qde Total de Faturas Pagas após Vencimento,COBRANÇA_VINDI_Qde Total de Faturas Inadimpletes,COBRANÇA_VINDI_Valor Total Inadimplência,COBRANÇA_VINDI_Qde Perfis de Pagamento Inativos
751,4141,1999-12-10,64,80,São Paulo,Ribeirão Preto,14096-260,79,2326,2023-03-15,...,NaN,NaN,8,9,Cartão de crédito,78.00,1,0,0,NaN


In [2838]:
df3 = preprocessing(df3_filtrado)

In [2839]:
df3.head()

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_id_gender,PESSOA_PIPEDRIVE_id_marrital_status,PESSOA_PIPEDRIVE_state,PESSOA_PIPEDRIVE_city,PESSOA_PIPEDRIVE_postal_code,PESSOA_PIPEDRIVE_contract_start_date,PESSOA_PIPEDRIVE_contract_end_date,PESSOA_PIPEDRIVE_Canal de Preferência,PESSOA_PIPEDRIVE_notes_count,...,WHOQOL_Psicológico_New,WHOQOL_Social_New,WHOQOL_Ambiental_New,COMUNICARE_Problemas Abertos Bool,TWILIO_Data Última Mensagens Inbound Recente,stay_time,last_stage_concluded,process_time,TWILIO_Data Última Mensagens Outbound Recente,TWILIO_Data Última Ligações Outbound Recente
906,5121,63,80,Amazonas,Novo Airão,69730-000,2023-06-12,NaN,0,2,...,5.0,4.0,3.0,0,0,179 days,Questionário,2023-08-31 20:33:36,0,0


In [2840]:
df3 = feature_engineering(df3)

(1, 45)


In [2841]:
df3.shape

(1, 45)

In [2842]:
df2.shape

(566, 88)

In [2843]:
df3.head()

,PESSOA_PIPEDRIVE_notes_count,PESSOA_PIPEDRIVE_done_activities_count,FUNIL_ONBOARDING_PIPEDRIVE_activities_count,ATENDIMENTOS_AGENDA_Faltas Todos Atendimento,ATENDIMENTOS_AGENDA_Qde Atendimento Médico,ATENDIMENTOS_AGENDA_Faltas Atendimento Médico,ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento,ATENDIMENTOS_AGENDA_Faltas Acolhimento,ATENDIMENTOS_AGENDA_Qde Psicoterapia,ATENDIMENTOS_AGENDA_Qde Prescrições,...,FUNIL_ASSINATURA_PIPEDRIVE_id_stage Binário,Status_80,Estado_Amazonas,PESSOA_PIPEDRIVE_city Codificada,status_won,lost_reason_Outro,canal_preferencia_0,Status_lost,lost_reason_Outro,stage_Questionário
906,2,9,5,0,0,0,1,0,0,0,...,0,True,True,1,True,True,True,True,True,True


In [2844]:
df3 = df3.drop(columns=["stay_time"])

In [2845]:
df3

,PESSOA_PIPEDRIVE_notes_count,PESSOA_PIPEDRIVE_done_activities_count,FUNIL_ONBOARDING_PIPEDRIVE_activities_count,ATENDIMENTOS_AGENDA_Faltas Todos Atendimento,ATENDIMENTOS_AGENDA_Qde Atendimento Médico,ATENDIMENTOS_AGENDA_Faltas Atendimento Médico,ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento,ATENDIMENTOS_AGENDA_Faltas Acolhimento,ATENDIMENTOS_AGENDA_Qde Psicoterapia,ATENDIMENTOS_AGENDA_Qde Prescrições,...,FUNIL_ASSINATURA_PIPEDRIVE_id_stage Binário,Status_80,Estado_Amazonas,PESSOA_PIPEDRIVE_city Codificada,status_won,lost_reason_Outro,canal_preferencia_0,Status_lost,lost_reason_Outro,stage_Questionário
906,2,9,5,0,0,0,1,0,0,0,...,0,True,True,1,True,True,True,True,True,True


In [2846]:
df2 = df2.drop(columns=["stay_time","PESSOA_PIPEDRIVE_id_person"])

In [2847]:
df2

,PESSOA_PIPEDRIVE_notes_count,PESSOA_PIPEDRIVE_done_activities_count,FUNIL_ONBOARDING_PIPEDRIVE_activities_count,ATENDIMENTOS_AGENDA_Faltas Todos Atendimento,ATENDIMENTOS_AGENDA_Qde Atendimento Médico,ATENDIMENTOS_AGENDA_Faltas Atendimento Médico,ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento,ATENDIMENTOS_AGENDA_Faltas Acolhimento,ATENDIMENTOS_AGENDA_Qde Psicoterapia,ATENDIMENTOS_AGENDA_Qde Prescrições,...,Status_open,Status_won,lost_reason_Outro.1,lost_reason_[Associade] Cancelou assinatura,lost_reason_[Onboarding] Não retornou aos contatos de resgate,lost_reason_[Onboarding] Não tem interesse em seguir nas etapas do onboarding,stage_Boas-vindas,stage_Não iniciado,stage_Primeira reunião,stage_Questionário
0,3,10,0,0,0,0,2,0,0,0.0,...,False,True,True,False,False,False,False,False,False,True
1,0,9,0,0,0,0,1,0,0,0.0,...,False,True,True,False,False,False,False,False,False,True
2,1,14,9,0,0,0,1,0,0,0.0,...,False,False,False,False,True,False,False,False,False,True
3,0,11,5,0,0,0,1,0,0,0.0,...,False,True,True,False,False,False,False,False,False,True
4,0,7,1,0,0,0,1,0,25,1.0,...,False,True,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,0,1,1,0,0,0,2,0,0,0.0,...,True,False,True,False,False,False,False,False,False,True
562,0,0,0,0,0,0,2,0,1,0.0,...,True,False,True,False,False,False,False,False,False,True
563,0,1,1,0,1,0,3,0,1,0.0,...,True,False,True,False,False,False,False,False,True,False
564,0,0,0,0,0,0,1,0,0,0.0,...,True,False,True,False,False,False,False,False,True,False


In [2848]:
df2 = pd.DataFrame(df2.iloc[50]).T

In [2849]:
df2.to_json( force_ascii=False, orient='records', index=False)

'[{"PESSOA_PIPEDRIVE_notes_count":0,"PESSOA_PIPEDRIVE_done_activities_count":7,"FUNIL_ONBOARDING_PIPEDRIVE_activities_count":3,"ATENDIMENTOS_AGENDA_Faltas Todos Atendimento":0,"ATENDIMENTOS_AGENDA_Qde Atendimento Médico":0,"ATENDIMENTOS_AGENDA_Faltas Atendimento Médico":0,"ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento":1,"ATENDIMENTOS_AGENDA_Faltas Acolhimento":0,"ATENDIMENTOS_AGENDA_Qde Psicoterapia":0,"ATENDIMENTOS_AGENDA_Qde Prescrições":0.0,"TWILIO_Mensagens Inbound":29,"TWILIO_Mensagens Outbound":42,"TWILIO_Ligações Outbound":2,"PESSOA_PIPEDRIVE_age":27,"PESSOA_PIPEDRIVE_has_public_health_plan":1,"PESSOA_PIPEDRIVE_tem_data_dataframe_de_termino_de_contrato":1,"PESSOA_PIPEDRIVE_Tem_Canal_de_Preferência":0,"PESSOA_PIPEDRIVE_has_notes":0,"WHOQOL_Físico_New":0.5414887639,"WHOQOL_Psicológico_New":0.7004690034,"WHOQOL_Social_New":0.4268099387,"WHOQOL_Ambiental_New":4.0,"COMUNICARE_Problemas Abertos Bool":1,"TWILIO_Data Última Mensagens Inbound Recente":0,"TWILIO_Data Última Mensagens 

In [2853]:
predicao = model.predict(dfppp)

print(predicao)

[152.42316506]
